In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:

data = np.loadtxt('crop.csv',delimiter=',',skiprows=1,dtype=str,encoding=None)
print(data.shape)
nitrogen = data[:, 0].astype(float)
phosphorus = data[:, 1].astype(float)
potassium = data[:, 2].astype(float)
temperature = data[:, 3].astype(float)
humidity = data[:, 4].astype(float)
ph_value = data[:, 5].astype(float)
rainfall = data[:, 6].astype(float)
y = data[:, 7]
class_map = {'Rice': 0,'Maize': 1,'ChickPea': 2,'KidneyBeans': 3,'PigeonPeas': 4,
             'MothBeans': 5,'MungBean': 6,'Blackgram': 7,'Lentil': 8,'Pomegranate': 9,
             'Banana': 10,'Mango': 11,'Grapes': 12,'Watermelon': 13,'Muskmelon': 14,
             'Apple': 15,'Orange': 16,'Papaya': 17,'Coconut': 18,'Cotton': 19,
             'Jute': 20,'Coffee': 21}
y_numeric = np.array([class_map[label] if label in class_map else label for label in y])
X = np.column_stack((nitrogen/100, phosphorus/100, potassium/100, temperature/100, humidity/100, ph_value/100, rainfall/100))

train_indices = [list(range(i, i + 80)) for i in range(0, 2200, 100)]
train_indices = [index for sublist in train_indices for index in sublist]

test_indices = [i for i in range(2200) if i not in train_indices]

X_train_manual = X[train_indices]
y_train_manual = y_numeric[train_indices]

X_test_manual = X[test_indices]
y_test_manual = y_numeric[test_indices]

nb_train = len(X_train_manual)
nb_test = len(y_test_manual)

train_x = np.reshape(X_train_manual,[nb_train,-1])
test_x = np.reshape(X_test_manual,[nb_test,-1])

learning_rate = 0.001
nb_epochs = 100
batch_size = 16

nb_input = 1*7
nb_hidden1 = 32
nb_hidden2 = 32
nb_classes = 22

np.random.seed(42)
tf.random.set_seed(42)

w = {
    '1': tf.Variable(tf.random.normal([nb_input, nb_hidden1], dtype=tf.float64)),
    '2': tf.Variable(tf.random.normal([nb_hidden1, nb_hidden2], dtype=tf.float64)),
    'out': tf.Variable(tf.random.normal([nb_hidden2, nb_classes], dtype=tf.float64))
}

b = {
    '1': tf.Variable(tf.random.normal([nb_hidden1], dtype=tf.float64)),
    '2': tf.Variable(tf.random.normal([nb_hidden2], dtype=tf.float64)),
    'out': tf.Variable(tf.random.normal([nb_classes], dtype=tf.float64))
}

f = {
    '1': tf.nn.relu,
    '2': tf.nn.relu,
    'out': tf.nn.softmax
}

def runNN(x):
    z1 = tf.add(tf.matmul(x, w['1']), b['1'])
    a1 = f['1'](z1)
    z2 = tf.add(tf.matmul(a1, w['2']), b['2'])
    a2 = f['2'](z2)
    z_out = tf.add(tf.matmul(a2, w['out']), b['out']) # a2 ovde!
    out = f['out'](z_out)

    pred = tf.argmax(out, 1)

    return pred, z_out

opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)

for epoch in range(nb_epochs):
    epoch_loss = 0
    nb_batches = int(nb_train / batch_size)
    #batch je podskup trening setova
    for i in range(nb_batches):
        x = train_x[i*batch_size : (i+1)*batch_size, :]
        y = y_train_manual[i*batch_size : (i+1)*batch_size]
        y_onehot = tf.one_hot(y, nb_classes)

        with tf.GradientTape() as tape:
            _, z_out = runNN(x)

            loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=z_out, labels=y_onehot))

        w1_g, w2_g, wout_g, b1_g, b2_g, bout_g = tape.gradient(loss, [w['1'], w['2'], w['out'], b['1'], b['2'], b['out']])

        opt.apply_gradients(zip([w1_g, w2_g, wout_g, b1_g, b2_g, bout_g], [w['1'], w['2'], w['out'], b['1'], b['2'], b['out']]))

        epoch_loss += loss

    # U svakoj epohi ispisujemo prosečan loss.
    epoch_loss /= nb_train
    print(f'Epoch: {epoch+1}/{nb_epochs}| Avg loss: {epoch_loss:.5f}')

# Test!
x = test_x
y = y_test_manual

pred, _ = runNN(x)
pred_correct = tf.equal(pred, y)
accuracy = tf.reduce_mean(tf.cast(pred_correct, tf.float64))

print(f'Test acc: {accuracy:.3f}')



(2200, 8)
Epoch: 1/100| Avg loss: 2.20318
Epoch: 2/100| Avg loss: 1.50684
Epoch: 3/100| Avg loss: 1.11558
Epoch: 4/100| Avg loss: 0.87966
Epoch: 5/100| Avg loss: 0.73006
Epoch: 6/100| Avg loss: 0.61903
Epoch: 7/100| Avg loss: 0.53142
Epoch: 8/100| Avg loss: 0.46530
Epoch: 9/100| Avg loss: 0.40968
Epoch: 10/100| Avg loss: 0.35632
Epoch: 11/100| Avg loss: 0.30576
Epoch: 12/100| Avg loss: 0.26208
Epoch: 13/100| Avg loss: 0.22146
Epoch: 14/100| Avg loss: 0.18637
Epoch: 15/100| Avg loss: 0.15966
Epoch: 16/100| Avg loss: 0.13987
Epoch: 17/100| Avg loss: 0.12413
Epoch: 18/100| Avg loss: 0.11122
Epoch: 19/100| Avg loss: 0.10068
Epoch: 20/100| Avg loss: 0.09204
Epoch: 21/100| Avg loss: 0.08477
Epoch: 22/100| Avg loss: 0.07853
Epoch: 23/100| Avg loss: 0.07320
Epoch: 24/100| Avg loss: 0.06841
Epoch: 25/100| Avg loss: 0.06414
Epoch: 26/100| Avg loss: 0.06028
Epoch: 27/100| Avg loss: 0.05668
Epoch: 28/100| Avg loss: 0.05341
Epoch: 29/100| Avg loss: 0.05041
Epoch: 30/100| Avg loss: 0.04755
Epoch: 31